## VacationPy

### Obervations





In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
import datetime

# Import API key
from api_key import g_key

ModuleNotFoundError: No module named 'gmaps'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
# Read in the city weather data file created in WeatherPy
city_data = pd.read_csv("../output_data/city_data.csv")

# Display the dataframe
city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kurumkan,54.3000,110.3000,40.50,59,0,3.78,RU,05-14-2021
1,Gorom-Gorom,14.4439,-0.2361,107.73,13,37,2.95,BF,05-14-2021
2,Opuwo,-18.0607,13.8400,80.73,20,8,4.36,NaN,05-14-2021
3,Te Anau,-45.4167,167.7167,46.98,88,50,3.11,NZ,05-14-2021
4,Jumla,29.2747,82.1838,51.53,81,42,1.81,NP,05-14-2021


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = city_data[["Latitude", "Longitude"]]

humidity = city_data_clean["Humidity (%)"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)
# Add layer
fig.add_layer(heat_layer)

# Display the figure
fig
#print(fig)

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values

In [5]:
#reduce cities to ideal conditions 
preferred_cities = city_data_clean.loc[(city_data_clean["Wind Speed (mph)"]<= 10) & (city_data_clean["Cloudiness (%)"] == 0) & 
                                (city_data_clean["Max Temp (F)"]>= 70) & (city_data_clean["Max Temp (F)"]<= 80)].dropna()

preferred_cities.head()

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
33,hobart,AU,2021-01-24 15:43:18,-42.8794,147.3294,0.0,49.0,70.268,4.63
47,new norfolk,AU,2021-01-24 15:43:19,-42.7826,147.0587,0.0,49.0,70.268,4.63
145,chuy,UY,2021-01-24 15:43:26,-33.6971,-53.4616,0.0,70.0,78.062,4.71
207,valdivia,CL,2021-01-24 15:41:40,-39.8142,-73.2459,0.0,27.0,79.070,6.17
245,salalah,OM,2021-01-24 15:43:33,17.0151,54.0924,0.0,37.0,71.870,4.63


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = preferred_cities
hotel_df['Hotel Name'] = " "
hotel_df.head()

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Hotel Name
33,hobart,AU,2021-01-24 15:43:18,-42.8794,147.3294,0.0,49.0,70.268,4.63,
47,new norfolk,AU,2021-01-24 15:43:19,-42.7826,147.0587,0.0,49.0,70.268,4.63,
145,chuy,UY,2021-01-24 15:43:26,-33.6971,-53.4616,0.0,70.0,78.062,4.71,
207,valdivia,CL,2021-01-24 15:41:40,-39.8142,-73.2459,0.0,27.0,79.070,6.17,
245,salalah,OM,2021-01-24 15:43:33,17.0151,54.0924,0.0,37.0,71.870,4.63,


In [7]:
# Search criteria
criteria_search = 'Hotel'
criteria_radius = 5000
criteria_type = 'lodging'

#Variable to keep track of response number
record = 0

for index, row in hotel_df.iterrows():
    # create a dictionary of parameters that need to be passed to the request
    params = {
        "location": f"{row['Latitude']}, {row['Longitude']}",
        "keyword": criteria_search,
        "radius": criteria_radius,
        "type": criteria_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make api requests using the parameters 
    response = requests.get(base_url, params=params)
    response_json = response.json()
    results = response_json['results']
    if len(results) > 0:
        name = response_json['results'][0]['name']
        hotel_df.iloc[record, -1] = name
    record = record + 1

nearest_hotel_preferred_cities = hotel_df
nearest_hotel_preferred_cities.head()

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Hotel Name
33,hobart,AU,2021-01-24 15:43:18,-42.8794,147.3294,0.0,49.0,70.268,4.63,Hotel Grand Chancellor Hobart
47,new norfolk,AU,2021-01-24 15:43:19,-42.7826,147.0587,0.0,49.0,70.268,4.63,
145,chuy,UY,2021-01-24 15:43:26,-33.6971,-53.4616,0.0,70.0,78.062,4.71,Bertelli Chuí Hotel
207,valdivia,CL,2021-01-24 15:41:40,-39.8142,-73.2459,0.0,27.0,79.070,6.17,Hotel Nueve Rios
245,salalah,OM,2021-01-24 15:43:33,17.0151,54.0924,0.0,37.0,71.870,4.63,HYATT SALALAH HOTEL


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in nearest_hotel_preferred_cities.iterrows()]
locations = nearest_hotel_preferred_cities[["Latitude", "Longitude"]]

In [9]:
# Add a marker layer on top of the heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer
fig.add_layer(marker_layer)

# Display figure
fig
#print(fig)

Figure(layout=FigureLayout(height='420px'))